# Subject Recognition CRF Tagger Model

In [1]:
import sys
sys.path.insert(0, '../../../allennlp')
sys.path.insert(0, '../../')

In [2]:
import os
from allennlp.common.params import Params

crf_tagger = {
    "dataset_reader": {
        "type": "sequence_tagging",
        "word_tag_delimiter": "/",
        "token_indexers": {
            "tokens": {
                "type": "single_id",
                "lowercase_tokens": True
            },
            "token_characters": {
                "type": "characters",
                "character_tokenizer": {
                    "end_tokens": [
                        "@@PADDING@@",
                        "@@PADDING@@",
                        "@@PADDING@@",
                        "@@PADDING@@"
                    ]
                }
            }
        }
    },
    # TODO: Update this to the location for subject name recognition training data
    "train_data_path": './../../data/subject_recognition/train.txt', 
    "validation_data_path": './../../data/subject_recognition/dev.txt',
    "model": {
        "type": "crf_tagger",
        "text_field_embedder": {
            "tokens": {
                "type": "embedding",
                "pretrained_file": "https://s3-us-west-2.amazonaws.com/allennlp/datasets/glove/glove.6B.100d.txt.gz",
                "embedding_dim": 100,
                "trainable": False
            },
            "token_characters": {
                "type": "character_encoding",
                "embedding": {
                    "embedding_dim": 16
                },
                "encoder": {
                    "type": "cnn",
                    "embedding_dim": 16,
                    "num_filters": 100,
                    "ngram_filter_sizes": [
                        5
                    ]
                },
                "dropout": 0.6
            }
        },
        "encoder": {
            "type": "lstm",
            "input_size": 200,
            "hidden_size": 300,
            "num_layers": 3,
            "dropout": 0.2,
            "bidirectional": True
        }
    },
    "iterator": {
        "type": "bucket",
        "sorting_keys": [
            [
                "tokens",
                "num_tokens"
            ]
        ],
        "batch_size": 16,
    },
    "trainer": {
        "optimizer": "adam",
        "num_epochs": 30,
        "grad_norm": 1.0,
        "cuda_device": 0,
        "patience": 10,
        "validation_metric": "+accuracy",
        "learning_rate_scheduler":  {
          "type": "reduce_on_plateau",
          "factor": 0.5,
          "mode": "max",
          "patience": 2,
        },
        "optimizer": {
          "type": "adam",
        }
    }
}

In [3]:
from lib.utils import config_logging
from lib.utils import new_experiment_folder
from allennlp.commands.train import train_model

config_logging()
experiment_folder = new_experiment_folder(label='subject_recognition', parent_directory='../../experiments/')
params = Params(crf_tagger)
print('Serialization Directory:', experiment_folder)
train_model(params=params, serialization_dir=experiment_folder)

Serialization Directory: ../../experiments/subject_recognition.02_01_19:36:36
[2018-02-01 19:36:36,418][MainProcess][allennlp.common.params][INFO] random_seed = 13370
[2018-02-01 19:36:36,418][MainProcess][allennlp.common.params][INFO] numpy_seed = 1337
[2018-02-01 19:36:36,419][MainProcess][allennlp.common.params][INFO] pytorch_seed = 133
[2018-02-01 19:36:36,420][MainProcess][allennlp.common.checks][INFO] Pytorch version: 0.3.0.post4
[2018-02-01 19:36:36,422][MainProcess][allennlp.common.params][INFO] dataset_reader.type = sequence_tagging
[2018-02-01 19:36:36,423][MainProcess][allennlp.common.params][INFO] dataset_reader.token_indexers.tokens.type = single_id
[2018-02-01 19:36:36,424][MainProcess][allennlp.common.params][INFO] dataset_reader.token_indexers.tokens.namespace = tokens
[2018-02-01 19:36:36,424][MainProcess][allennlp.common.params][INFO] dataset_reader.token_indexers.tokens.lowercase_tokens = True
[2018-02-01 19:36:36,425][MainProcess][allennlp.common.params][INFO] datas

74520it [00:02, 31022.92it/s]

[2018-02-01 19:36:38,909][MainProcess][allennlp.common.params][INFO] validation_data_path = ./../../data/subject_recognition/dev.txt
[2018-02-01 19:36:38,910][MainProcess][allennlp.commands.train][INFO] Reading validation data from ./../../data/subject_recognition/dev.txt
[2018-02-01 19:36:38,911][MainProcess][allennlp.data.dataset_readers.sequence_tagging][INFO] Reading instances from lines in file at: ./../../data/subject_recognition/dev.txt



10648it [00:00, 49433.40it/s]

[2018-02-01 19:36:39,140][MainProcess][allennlp.common.params][INFO] test_data_path = None
[2018-02-01 19:36:39,140][MainProcess][allennlp.commands.train][INFO] Creating a vocabulary using train, validation data.


...

token_accuracy: 1.00, accuracy: 1.00, loss: 0.02 ||: 100%|##########| 4658/4658 [01:50<00:00, 42.25it/s]

[2018-02-01 20:28:35,205][MainProcess][allennlp.training.trainer][INFO] Validating



token_accuracy: 0.99, accuracy: 0.95, loss: 19.38 ||: 100%|##########| 666/666 [00:08<00:00, 81.13it/s]


[2018-02-01 20:28:43,416][MainProcess][allennlp.training.trainer][INFO] Ran out of patience.  Stopping training.
[2018-02-01 20:28:43,417][MainProcess][allennlp.models.archival][INFO] archiving weights and vocabulary to ../../experiments/subject_recognition.02_01_19:36:36/model.tar.gz


CrfTagger(
  (text_field_embedder): BasicTextFieldEmbedder(
    (token_embedder_tokens): Embedding(
    )
    (token_embedder_token_characters): TokenCharactersEncoder(
      (_embedding): TimeDistributed(
        (_module): Embedding(
        )
      )
      (_encoder): TimeDistributed(
        (_module): CnnEncoder(
          (_activation): ReLU()
          (conv_layer_0): Conv1d (16, 100, kernel_size=(5,), stride=(1,))
        )
      )
      (_dropout): Dropout(p=0.6)
    )
  )
  (encoder): PytorchSeq2SeqWrapper(
    (_module): LSTM(200, 300, num_layers=3, batch_first=True, dropout=0.2, bidirectional=True)
  )
  (tag_projection_layer): TimeDistributed(
    (_module): Linear(in_features=600, out_features=2)
  )
  (crf): ConditionalRandomField(
  )
)